# Interactive chat

In [ ]:
import torch
import os
import json
import sys
import numpy as np


sys.path.append('.')
sys.path.append('..')

from utils.probing_utils import *
from utils.inference_utils import *

torch.set_float32_matmul_precision('high')

In [ ]:
CHAT_MODEL_NAME = "Qwen/Qwen3-32B"
MODEL_READABLE = "Qwen 3 32B"
OUTPUT_DIR = "./results/qwen-3-32b/interactive"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:

model = load_vllm_model(CHAT_MODEL_NAME, max_model_len=40960, tensor_parallel_size=2)
print(f"✅ Model {MODEL_READABLE} loaded successfully!")


In [ ]:
conversation_history = []


In [ ]:

def chat_interactive(message, show_history=False, return_response=False):
    """Interactive chat function"""
    global conversation_history
    response, conversation_history = continue_conversation(
        model, 
        conversation_history, 
        message,
        max_tokens=3000,
        temperature=0.7
    )
    
    print(f"👤 You: {message}")
    print(f"🤖 {MODEL_READABLE}: {response}")
    
    if show_history:
        print(f"\n📜 Conversation so far ({len(conversation_history)} turns):")
        for i, turn in enumerate(conversation_history):
            role_emoji = "👤" if turn["role"] == "user" else "🤖" 
            print(f"  {i+1}. {role_emoji} {turn['content'][:100]}...")
    
    # Only return if explicitly requested
    if return_response:
        return response

# Advanced conversation analysis tools

def save_conversation(filename):
    """Save the current conversation to a file"""
    if not conversation_history:
        print("No conversation to save!")
        return
    
    conversation_data = {
        "model": CHAT_MODEL_NAME,
        "turns": len(conversation_history),
        "conversation": conversation_history
    }
    
    with open(f"{OUTPUT_DIR}/{filename}", 'w') as f:
        json.dump(conversation_data, f, indent=2)
    
    print(f"💾 Conversation saved to: {filename}")
    return filename

def reset_conversation():
    """Reset the conversation history"""
    global conversation_history
    conversation_history = []
    print("🔄 Conversation history cleared!")


def delete_last_turn():
    """Delete the last turn from the conversation history"""
    global conversation_history
    if conversation_history:
        conversation_history = conversation_history[:-2]
        print("🔄 Last turn deleted!")
    else:
        print("No conversation to delete!")

In [ ]:
save_conversation("therapy.json")

In [ ]:
delete_last_turn()

In [ ]:
model.close()